In [1]:
import numpy as np
import matplotlib.pyplot as plt
import deepdish as dd

### load data

In [2]:
vector_field = dd.io.load('vector_fields.h5')

In [3]:
vector_field.shape

(105, 4, 840, 840)

### create abs distance matrices

In [4]:
abs_distance = np.zeros_like(vector_field)

### 4 channels meaning: (dxh, dyh, dxt, dyt)

In [5]:
abs_distance.shape

(105, 4, 840, 840)

### compute abs distance

In [6]:
def compute_abs_dist(abs_distance):
    pad_distance = np.arange(abs_distance.shape[2])
    n_repeat = abs_distance.shape[2]
    n_data = abs_distance.shape[0]
    
    for i in range(n_data):
        abs_distance[i][0] = abs_distance[i][2] = np.repeat(pad_distance.reshape(-1,1), n_repeat, axis=1)
        abs_distance[i][1] = abs_distance[i][3] = np.repeat(pad_distance.reshape(1,-1), n_repeat, axis=0)
    
    return abs_distance

In [7]:
abs_distance = compute_abs_dist(abs_distance)

### add abs distance to vector field

In [8]:
abs_vf = vector_field + abs_distance

In [13]:
abs_vf[0]

array([[[  527.,   526.,   525., ...,  -212.,  -213.,  -214.],
        [  528.,   527.,   526., ...,  -211.,  -212.,  -213.],
        [  529.,   528.,   527., ...,  -210.,  -211.,  -212.],
        ..., 
        [ 1364.,  1363.,  1362., ...,   625.,   624.,   623.],
        [ 1365.,  1364.,  1363., ...,   626.,   625.,   624.],
        [ 1366.,  1365.,  1364., ...,   627.,   626.,   625.]],

       [[  304.,   305.,   306., ...,  1485.,  1486.,  1487.],
        [  303.,   304.,   305., ...,  1484.,  1485.,  1486.],
        [  302.,   303.,   304., ...,  1483.,  1484.,  1485.],
        ..., 
        [ -533.,  -532.,  -531., ...,   648.,   649.,   650.],
        [ -534.,  -533.,  -532., ...,   647.,   648.,   649.],
        [ -535.,  -534.,  -533., ...,   646.,   647.,   648.]],

       [[  612.,   611.,   610., ...,  -142.,  -143.,  -144.],
        [  613.,   612.,   611., ...,  -141.,  -142.,  -143.],
        [  614.,   613.,   612., ...,  -140.,  -141.,  -142.],
        ..., 
        [

In [14]:
vector_field[0]

array([[[ 527.,  526.,  525., ..., -212., -213., -214.],
        [ 527.,  526.,  525., ..., -212., -213., -214.],
        [ 527.,  526.,  525., ..., -212., -213., -214.],
        ..., 
        [ 527.,  526.,  525., ..., -212., -213., -214.],
        [ 527.,  526.,  525., ..., -212., -213., -214.],
        [ 527.,  526.,  525., ..., -212., -213., -214.]],

       [[ 304.,  304.,  304., ...,  648.,  648.,  648.],
        [ 303.,  303.,  303., ...,  647.,  647.,  647.],
        [ 302.,  302.,  302., ...,  646.,  646.,  646.],
        ..., 
        [-533., -533., -533., ..., -189., -189., -189.],
        [-534., -534., -534., ..., -190., -190., -190.],
        [-535., -535., -535., ..., -191., -191., -191.]],

       [[ 612.,  611.,  610., ..., -142., -143., -144.],
        [ 612.,  611.,  610., ..., -142., -143., -144.],
        [ 612.,  611.,  610., ..., -142., -143., -144.],
        ..., 
        [ 612.,  611.,  610., ..., -142., -143., -144.],
        [ 612.,  611.,  610., ..., -142., 

In [16]:
vector_field[0] + abs_distance[0]

array([[[  527.,   526.,   525., ...,  -212.,  -213.,  -214.],
        [  528.,   527.,   526., ...,  -211.,  -212.,  -213.],
        [  529.,   528.,   527., ...,  -210.,  -211.,  -212.],
        ..., 
        [ 1364.,  1363.,  1362., ...,   625.,   624.,   623.],
        [ 1365.,  1364.,  1363., ...,   626.,   625.,   624.],
        [ 1366.,  1365.,  1364., ...,   627.,   626.,   625.]],

       [[  304.,   305.,   306., ...,  1485.,  1486.,  1487.],
        [  303.,   304.,   305., ...,  1484.,  1485.,  1486.],
        [  302.,   303.,   304., ...,  1483.,  1484.,  1485.],
        ..., 
        [ -533.,  -532.,  -531., ...,   648.,   649.,   650.],
        [ -534.,  -533.,  -532., ...,   647.,   648.,   649.],
        [ -535.,  -534.,  -533., ...,   646.,   647.,   648.]],

       [[  612.,   611.,   610., ...,  -142.,  -143.,  -144.],
        [  613.,   612.,   611., ...,  -141.,  -142.,  -143.],
        [  614.,   613.,   612., ...,  -140.,  -141.,  -142.],
        ..., 
        [